In [ ]:
# Make it compatiple with yolo 12
# Start working on the pedal and stick tracking

In [ ]:
# Copied from the code given by Roboflow for training

Upload images

In [3]:
from roboflow import Roboflow

# Initialize the Roboflow object with your API key
with open('ROBOFLOW_API_KEY.txt', 'r') as file:
    ROBOFLOW_API_KEY = file.read()

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Retrieve your current workspace and project name
print(rf.workspace())

# Specify the project for upload
# let's you have a project at https://app.roboflow.com/my-workspace/my-project
workspaceId = "seniordesign-neufu"
projectId = 'pedals_stick'
project = rf.workspace(workspaceId).project(projectId)

# Upload the image to your project
project.upload("webf2", batch_name="Nov 17 Flight 2")

"""
Optional Parameters:
- num_retry_uploads: Number of retries for uploading the image in case of failure.
- batch_name: Upload the image to a specific batch.
- split: Upload the image to a specific split.
- tag: Store metadata as a tag on the image.
- sequence_number: [Optional] If you want to keep the order of your images in the dataset, pass sequence_number and sequence_size..
- sequence_size: [Optional] The total number of images in the sequence. Defaults to 100,000 if not set.
"""
'''
project.upload(
    image_path="UPLOAD_IMAGE.jpg",
    batch_name="YOUR_BATCH_NAME",
    split="train",
    num_retry_uploads=3,
    tag_names=["YOUR_TAG_NAME"],
    sequence_number=99,
    sequence_size=100
)
'''

loading Roboflow workspace...
{
  "name": "SeniorDesign",
  "url": "seniordesign-neufu",
  "projects": [
    "seniordesign-neufu/dials-3dlwi",
    "seniordesign-neufu/pedals_stick"
  ]
}
loading Roboflow workspace...
loading Roboflow project...
[ webf2/frame_0000.png ] was uploaded succesfully.
[ webf2/frame_0030.png ] was uploaded succesfully.
[ webf2/frame_0060.png ] was uploaded succesfully.
[ webf2/frame_0090.png ] was uploaded succesfully.
[ webf2/frame_0120.png ] was uploaded succesfully.
[ webf2/frame_0150.png ] was uploaded succesfully.
[ webf2/frame_0180.png ] was uploaded succesfully.
[ webf2/frame_0210.png ] was uploaded succesfully.
[ webf2/frame_0240.png ] was uploaded succesfully.
[ webf2/frame_0270.png ] was uploaded succesfully.
[ webf2/frame_0300.png ] was uploaded succesfully.
[ webf2/frame_0330.png ] was uploaded succesfully.
[ webf2/frame_0360.png ] was uploaded succesfully.
[ webf2/frame_0390.png ] was uploaded succesfully.
[ webf2/frame_0420.png ] was uploaded suc

'\nproject.upload(\n    image_path="UPLOAD_IMAGE.jpg",\n    batch_name="YOUR_BATCH_NAME",\n    split="train",\n    num_retry_uploads=3,\n    tag_names=["YOUR_TAG_NAME"],\n    sequence_number=99,\n    sequence_size=100\n)\n'

Set up

In [ ]:
#%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

In [ ]:
from roboflow import Roboflow
import os

home_dir = "C:/Users/Alex/Desktop/Dial Detection"
os.chdir(home_dir)

with open('ROBOFLOW_API_KEY.txt', 'r') as file:
    ROBOFLOW_API_KEY = file.read()

path = "datasets"

#os.mkdir(path)

os.chdir(path)

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("seniordesign-neufu")
project = workspace.project("dials-3dlwi") # Need to change the project
version = project.version(2) # Change this to be the dataset version number
dataset = version.download("yolov12")

Training

In [ ]:
from ultralytics import YOLO
import torch
import os

os.chdir(home_dir)
print(os.getcwd())

# Using pretrained weights, make sure this is correct!
model = YOLO('models/yolo12x.pt')
model.train(data=dataset.location + "/data.yaml", epochs=100, imgsz=640, batch=0.8, project="saved_models", name="m", plots=True)

In [ ]:
#print(home_dir + "/runs/detect/train/")

Visualizing

In [ ]:
# Gets the newest directory, helps with visualization
import os

# Written by bing copliot
def get_newest_directory(path):
    # List all directories in the given path
    directories = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    
    # Sort directories by creation time
    newest_directory = max(directories, key=os.path.getctime)
    
    return newest_directory

# Example usage
path = home_dir + '/saved_models/'
path = get_newest_directory(path)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=path + '/confusion_matrix.png' , width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=path + '/results.png', width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=path + '/val_batch0_pred.jpg', width=600)

Deploy the model

In [ ]:
project.version(dataset.version).deploy(model_type="yolov12", model_path=path)